<a href="https://colab.research.google.com/github/Anirudh-R-1201/RecommendationSystem/blob/master/KeyPhraseExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving PlayStoreReviews_India_Aug8.xlsx to PlayStoreReviews_India_Aug8.xlsx


In [ ]:
import pandas as pd 
import io 
df = pd.read_excel(io.BytesIO(uploaded['PlayStoreReviews_India_Aug8.xlsx'])) 
df2 = df

In [ ]:
count = df.groupby("Review", as_index=False).count()
mean = df.groupby("Review", as_index=False).mean()
df_temp = pd.merge(df, count, how='right', on=['Review'])
df_temp = pd.merge(df_temp, mean, how='right', on=['Review'])
df_temp[:5]

,Country_x,App_x,Date_x,Customer_x,Rating_x,Review,Country_y,App_y,Date_y,Customer_y,Rating_y,Rating
0,India,HSBC,"July 21, 2020",Sam Joseph GodMan.J,1,I never used a worst banking app like this. I ...,1,1,1,1,1,1
1,India,HSBC,"July 14, 2020",Abhinav Saraswat,1,"Worst app i ever seen, goverment bank SBI Appl...",1,1,1,1,1,1
2,India,HSBC,"July 19, 2020",Vikas Kumar,1,Worst app ever i am using this app from last 1...,1,1,1,1,1,1
3,India,HSBC,"June 4, 2020",Kalakar Baba,1,Application working fine but it is features le...,1,1,1,1,1,1
4,India,HSBC,"May 26, 2020",anup kalshetty,1,Worst banking app if compared to other apps li...,1,1,1,1,1,1


In [ ]:
df1 = df_temp.groupby("App_x", as_index=False).mean()
combine_summary = df_temp.groupby("App_x")["Review"].apply(list)
combine_summary = pd.DataFrame(combine_summary)
combine_summary.to_csv("combine_summary.csv")
df1

,App_x,Rating_x,Country_y,App_y,Date_y,Customer_y,Rating_y,Rating
0,Axis Bank,2.365,1.00,1.00,1.00,1.00,1.00,2.365
1,Citi,2.365,1.00,1.00,1.00,1.00,1.00,2.365
2,Freecharge,2.170,1.00,1.00,1.00,1.00,1.00,2.170
3,HDFC Bank,1.310,1.01,1.01,1.01,1.01,1.01,1.310
4,HSBC,1.725,1.00,1.00,1.00,1.00,1.00,1.725
5,ICICI Bank,1.890,1.00,1.00,1.00,1.00,1.00,1.890
6,Kotak Bank,2.345,1.00,1.00,1.00,1.00,1.00,2.345
7,Mobikwik,2.090,1.00,1.00,1.00,1.00,1.00,2.090
8,PayTM,1.945,1.00,1.00,1.00,1.00,1.00,1.945
9,PayZapp,1.600,1.00,1.00,1.00,1.00,1.00,1.600


In [ ]:
df2 = pd.read_csv("combine_summary.csv")
df2 = pd.merge(df2, df_temp, on="App_x", how='inner')
df2

,App_x,Review_x,Country_x,Date_x,Customer_x,Rating_x,Review_y,Country_y,App_y,Date_y,Customer_y,Rating_y,Rating
0,Axis Bank,"[""The app is all good. I am using the account ...",India,5 August 2020,keshika khatri,4,The app is all good. I am using the account si...,1,1,1,1,1,4
1,Axis Bank,"[""The app is all good. I am using the account ...",India,5 August 2020,Jishant Singh,1,Horrible. Using credit card section. 1. Pay no...,1,1,1,1,1,1
2,Axis Bank,"[""The app is all good. I am using the account ...",India,5 August 2020,Pradeep S M,4,User Friendly But... Before corona time I can ...,1,1,1,1,1,4
3,Axis Bank,"[""The app is all good. I am using the account ...",India,28 July 2020,Joel George,5,Edit 2 - Excellent app. Serves the purpose bri...,1,1,1,1,1,5
4,Axis Bank,"[""The app is all good. I am using the account ...",India,21 July 2020,lavish mehta,1,"I am still getting this stupid ""System under m...",1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Yono,['The app is not at all good in its latest ver...,India,"July 31, 2020",Lilly Anto,1,How many times I tried to login and not connec...,1,1,1,1,1,1
2996,Yono,['The app is not at all good in its latest ver...,India,"June 20, 2020",Chippigiri Lakshmikantha,4,Yono pay section. Transfer to other bank accou...,1,1,1,1,1,4
2997,Yono,['The app is not at all good in its latest ver...,India,"July 2, 2020",JAYPRAKASH JOSHI,4,But bill payment system takes 3 days. While ot...,1,1,1,1,1,4
2998,Yono,['The app is not at all good in its latest ver...,India,"June 25, 2020",Sunil Kumar Pandit,1,my mobile is not rooted / using virtual enviro...,1,1,1,1,1,1


In [ ]:
df2 = df2.drop_duplicates(['App_x'], keep='last')
df2 = df2.reset_index()
df2.head()

,index,App_x,Review_x,Country_x,Date_x,Customer_x,Rating_x,Review_y,Country_y,App_y,Date_y,Customer_y,Rating_y,Rating
0,199,Axis Bank,"[""The app is all good. I am using the account ...",India,31 July 2020,Amish Darbar,5,App is very good work but bank need update ATM...,1,1,1,1,1,5
1,399,Citi,['Experience is good. Easy to use due to norma...,India,19 May 2020,Gaurav Dutta,2,App is not very useful. Transactions update de...,1,1,1,1,1,2
2,599,Freecharge,['Far better than prior years. Thanks to its r...,India,"June 14, 2020",Mohamed Nizam,4,Colourful app without all the requirements and...,1,1,1,1,1,4
3,799,HDFC Bank,"[""Poor app. Irritating errors. Unable to load ...",India,21 June 2020,Soham Pradhan,1,Not at all user friendly. 'Card' section is un...,1,1,1,1,1,1
4,999,HSBC,['I never used a worst banking app like this. ...,India,"July 13, 2020",Javid Rather,1,"I think worst banking experience , app is a di...",1,1,1,1,1,1


PreTrained BERT Model:

In [ ]:
df["translated_text"][4]

'The food is very tasty, the place is small more very cozy, there are combos and promotions that are very attractive. Super recommend the Yakisoba is a delight'

In [ ]:
df1 = df

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-bafakw0j
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-bafakw0j
     |████████████████████████████████| 245kB 5.4MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8759829 sha256=391f6dd4b8f3cc2f2c91e8b0e8a35fb49e62b1466128a6068f3fe710793071a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-y8hdc06x/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
import re
import pke
import datetime
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def extract_keyphrases(caption, n):
    extractor = pke.unsupervised.TextRank() 
    extractor.load_document(caption)
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=n, stemming=False)
    phrases = []
    for i in range(len(keyphrases)):
      phrases.append(keyphrases[i])
    return(phrases)
a = datetime.datetime.now()    
df2['tag_BERT'] = df2.apply(lambda row: (extract_keyphrases(row['Review'],3)),axis=1)
b = datetime.datetime.now()
df2.head()

,Country,App,Date,Customer,Rating,Review,tag_BERT
0,India,HSBC,"July 21, 2020",Sam Joseph GodMan.J,1,I never used a worst banking app like this. I ...,"[(worst banking app, 0.14513851630227806), (di..."
1,India,HSBC,"July 14, 2020",Abhinav Saraswat,1,"Worst app i ever seen, goverment bank SBI Appl...","[(goverment bank sbi application, 0.3741709727..."
2,India,HSBC,"July 19, 2020",Vikas Kumar,1,Worst app ever i am using this app from last 1...,"[(credit card transactions, 0.252101130336128)..."
3,India,HSBC,"June 4, 2020",Kalakar Baba,1,Application working fine but it is features le...,"[(monthly view statement option, 0.22140741177..."
4,India,HSBC,"May 26, 2020",anup kalshetty,1,Worst banking app if compared to other apps li...,"[(worst banking app, 0.30619885793389157), (su..."


In [ ]:
print("PKE Time duration for 500 reviews")
print(b-a)
pke_time = b-a

PKE Time duration for 500 reviews
0:00:07.969475


In [ ]:
df_tagsPKE = df1[["product_name" , "crayon_product_id" , "Abstract_Keyphrases"]]

In [ ]:
df_tagsPKE

,product_name,crayon_product_id,Abstract_Keyphrases
0,Baixocesso Pizzaria,R-102001000-200859931,"[cold beer, good service, various flavors]"
1,California Coffee,R-102001000-201059308,"[iced coffees, california coffee, day rush]"
2,"Vikings - Steaks, Seafood & Sandwiches",R-102001000-200779746,"[family, friends, weekend]"
3,Tadaki Estrelas,R-102001000-202786436,"[well varied menu, simple items, minutes]"
4,O Japa - Rosario,R-102001000-202829085,"[delight, yakisoba, attractive]"
...,...,...,...
495,Kimura Culinaria Japonesa,R-102001000-201154179,"[first ones, second round, items]"
496,Escondidinho,R-102001000-201843610,"[dry meat hiding place, great request, cozy en..."
497,Total Requinte,R-102001000-203601308,"[good service, good restaurant, delicious dess..."
498,Lagosta Carioca,R-102001000-200468358,"[good variety, main attraction, various types]"


RAKEing Algorithm

In [ ]:
df2 = df

In [ ]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=74446351448c0adc0fff0ededf9b964b9fa3ee8c6313635fe27ed4a48758f572
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [ ]:
from rake_nltk import Rake,Metric
r = Rake(min_length=2, max_length=3,ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO)

In [ ]:
def rake_keyphrases(content , n):
  tag = []
  r.extract_keywords_from_text(content)
  tag = r.get_ranked_phrases_with_scores()
  return tag[:n]
a = datetime.datetime.now()   
df2['tag_RAKE'] = df2.apply(lambda row: (rake_keyphrases(row['Review'],3)),axis=1)
b = datetime.datetime.now() 

print("RAKE Time taken for 500 reviews")
print(b-a)

RAKE Time taken for 500 reviews
0:00:00.950424


In [ ]:
df2

,Country,App,Date,Customer,Rating,Review,tag_BERT,tag_RAKE
0,India,HSBC,"July 21, 2020",Sam Joseph GodMan.J,1,I never used a worst banking app like this. I ...,"[(worst banking app, 0.14513851630227806), (di...","[(9.0, wireless payment ).), (9.0, even intern..."
1,India,HSBC,"July 14, 2020",Abhinav Saraswat,1,"Worst app i ever seen, goverment bank SBI Appl...","[(goverment bank sbi application, 0.3741709727...","[(8.5, full functional ????), (8.3333333333333..."
2,India,HSBC,"July 19, 2020",Vikas Kumar,1,Worst app ever i am using this app from last 1...,"[(credit card transactions, 0.252101130336128)...","[(9.0, worst app ever), (9.0, last 1 year), (9..."
3,India,HSBC,"June 4, 2020",Kalakar Baba,1,Application working fine but it is features le...,"[(monthly view statement option, 0.22140741177...","[(9.0, need lot improvememt), (9.0, every bank..."
4,India,HSBC,"May 26, 2020",anup kalshetty,1,Worst banking app if compared to other apps li...,"[(worst banking app, 0.30619885793389157), (su...","[(9.0, apps like hdfc), (7.833333333333334, wo..."
...,...,...,...,...,...,...,...,...
2995,India,Mobikwik,"July 26, 2020",Rajat Raj,5,Very convenient for all payments and redemptio...,"[(payback points, 0.7689600521483594), (cash, ...","[(4.0, payback points)]"
2996,India,Mobikwik,"August 4, 2020",Abhijit Metla,5,"Nice and quick payment app, made in India also...","[(quick payment app, 0.5504587355963303), (sup...","[(9.0, supercash policy changed), (9.0, quick ..."
2997,India,Mobikwik,"July 16, 2020",manik sharma,1,This is a fake app for loan. They dont provide...,"[(credit riak score, 0.41094070872828775), (cr...","[(9.0, dont waste time), (8.5, dont provide lo..."
2998,India,Mobikwik,"May 22, 2020",Pulkit Sharma Premium Videos,1,There is no customer care phone number. Only e...,"[(customer care phone number, 0.20131302911925...","[(9.0, sent several messages), (9.0, full kyc ..."


In [ ]:
df2 = df2[["App","Review","tag_BERT","tag_RAKE","tag_graph"]]

In [ ]:
df2

,App_x,Review_x,tag_BERT,tag_RAKE,tag_graph
0,Axis Bank,"[""The app is all good. I am using the account ...","[(update atm center, 0.410958984109589), (bad ...","[(4.0, good work), (4.0, bad condition)]","[(app, 0.04318062149559546), (bank app, 0.0290..."
1,Citi,['Experience is good. Easy to use due to norma...,"[(transactions update delay, 0.387096834193536...","[(9.0, transactions update delay), (4.0, tires...","[(app, 0.03550784790473474), (citi app, 0.0213..."
2,Freecharge,['Far better than prior years. Thanks to its r...,"[(account section, 0.4493875273271151), (colou...","[(9.0, colourful app without), (9.0, account s...","[(app, 0.04823647803681848), (freecharge app, ..."
3,HDFC Bank,"[""Poor app. Irritating errors. Unable to load ...","[(app development n design, 0.1851860951851851...","[(9.0, see required info), (4.0, user friendly...","[(app, 0.04476319652826501), (hdfc app, 0.0284..."
4,HSBC,['I never used a worst banking app like this. ...,"[(worst banking experience, 0.8333333533290622...",[],"[(app, 0.04949935894456424), (hsbc app, 0.0329..."
5,ICICI Bank,['The latest update has slowed the app drastic...,"[(customer care, 0.2285419631516695), (bad exp...","[(9.0, bad experience even), (4.0, working goo...","[(app, 0.04383156141949158), (app issue, 0.027..."
6,Kotak Bank,"[""Im unable to login from the last one month. ...","[(quick transactions, 0.6241118475323161), (fr...","[(4.0, user friendly), (4.0, stands rectified)...","[(app, 0.03589299101903011), (bank app, 0.0262..."
7,Mobikwik,['It seems there is a serious issue with this ...,"[(great update, 0.1265121173639213), (pdf form...","[(9.0, long time ....), (9.0, great update ..)...","[(app, 0.03958099540187532), (mobikwik app, 0...."
8,PayTM,"[""Very bad experience. Quite for forever Many ...","[(longer time duration, 0.8695652273912978), (...","[(9.0, always shows error)]","[(app, 0.03476398731436446), (paytm app, 0.028..."
9,PayZapp,['app is not opening in android 10 device. my ...,"[(other online purchase, 0.26431756061445166),...","[(9.0, give 1 star), (9.0, give 0 star), (8.5,...","[(app, 0.039180220047305016), (payment app, 0...."


In [ ]:
keyphrase["PKETag"] = df_tagsPKE["Abstract_Keyphrases"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
keyphrase

,product_name,RAKEphrases,PKETag
0,Baixocesso Pizzaria,"[various flavors, right point, pizza served]","[cold beer, good service, various flavors]"
1,California Coffee,"[iced coffees, good variety, day well]","[iced coffees, california coffee, day rush]"
2,"Vikings - Steaks, Seafood & Sandwiches",[],"[family, friends, weekend]"
3,Tadaki Estrelas,"[well varied menu, well prepared, simple items]","[well varied menu, simple items, minutes]"
4,O Japa - Rosario,[super recommend],"[delight, yakisoba, attractive]"
...,...,...,...
495,Kimura Culinaria Japonesa,"[already eating dessert, second round, pieces ...","[first ones, second round, items]"
496,Escondidinho,"[great request, good food, fast care]","[dry meat hiding place, great request, cozy en..."
497,Total Requinte,"[price within, good service, good restaurant]","[good service, good restaurant, delicious dess..."
498,Lagosta Carioca,"[various types, main attraction, good variety]","[good variety, main attraction, various types]"


GRAPH BASED

In [ ]:
df['product_name'].value_counts()[:5].sort_values(ascending=False)

Subway                     10
Cacau Show                  7
Chocolates Brasil Cacau     5
Sal Gastronomia             4
Rei do Mate                 4
Name: product_name, dtype: int64

In [ ]:
df3 = df

In [ ]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, nltk, string

    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text)))

    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def score_keyphrases_by_textrank(text, n_keywords=0.5):
    from itertools import takewhile, tee#, izip
    import networkx, nltk
    
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]
    candidates = extract_candidate_words(text)
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates))

    def pairwise(iterable):
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
        
    for w1, w2 in pairwise(candidates):
        if w2:
            graph.add_edge(*sorted([w1, w2]))
    ranks = networkx.pagerank(graph)
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            j = i + len(kp_words)
    
    return sorted(keyphrases.items(), key=lambda x: x[1], reverse=True)

In [ ]:
import datetime
a = datetime.datetime.now() 

df2['tag_graph'] = df2.apply(lambda row: (score_keyphrases_by_textrank(row['Review'])),axis=1)
b = datetime.datetime.now() 
print(b-a)

0:00:24.556555


In [ ]:
df2.to_excel("tags.xlsx")
from google.colab import files
files.download("tags.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df2.tail()

,App,Review,tag_BERT,tag_RAKE,tag_graph
2995,Mobikwik,Very convenient for all payments and redemptio...,"[(payback points, 0.7689600521483594), (cash, ...","[(4.0, payback points)]","[(payments, 0.19939773459751797), (payback poi..."
2996,Mobikwik,"Nice and quick payment app, made in India also...","[(quick payment app, 0.5504587355963303), (sup...","[(9.0, supercash policy changed), (9.0, quick ...","[(quick payment, 0.1421904374218091), (superca..."
2997,Mobikwik,This is a fake app for loan. They dont provide...,"[(credit riak score, 0.41094070872828775), (cr...","[(9.0, dont waste time), (8.5, dont provide lo...","[(loan, 0.14429135857755243), (provode loan, 0..."
2998,Mobikwik,There is no customer care phone number. Only e...,"[(customer care phone number, 0.20131302911925...","[(9.0, sent several messages), (9.0, full kyc ...","[(app, 0.06502933142602284), (error, 0.0633131..."
2999,Mobikwik,Using this App again after long time....This a...,"[(great update, 0.1265121173639213), (pdf form...","[(9.0, long time ....), (9.0, great update ..)...","[(app, 0.07306122837355379), (great, 0.0411169..."


In [ ]:
#keyphrase["tag_graph"] = df2["tag_graph"]
for j in range(len(df2)):
  tag = []
  for i in df2["tag_graph"][j]:
    tag.append(i[0])
    df2["tag_graph"][j] = tag
  #df2["tag_graph"][0][0][0]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df2

,App_x,Review_x,tag_BERT,tag_RAKE,tag_graph
0,Axis Bank,"[""The app is all good. I am using the account ...","[update atm center, bad condition, good work]","[good work, bad condition]","[app, bank app, axis app, mobile app, good app..."
1,Citi,['Experience is good. Easy to use due to norma...,"[transactions update delay, tiresome process, ...","[transactions update delay, tiresome process, ...","[app, citi app, good app, app unable, citibank..."
2,Freecharge,['Far better than prior years. Thanks to its r...,"[account section, colourful app, better]","[colourful app without, account section could]","[app, freecharge app, good app, payment app, u..."
3,HDFC Bank,"[""Poor app. Irritating errors. Unable to load ...","[app development n design, savings bank accoun...","[see required info, user friendly, separate se...","[app, hdfc app, bank app, good app, new app, o..."
4,HSBC,['I never used a worst banking app like this. ...,"[worst banking experience, imps, option]",[],"[app, hsbc app, banking app, new app, worst ap..."
5,ICICI Bank,['The latest update has slowed the app drastic...,"[customer care, bad experience, recent months]","[bad experience even, working good, session ex...","[app, app issue, banking app, good app, app ve..."
6,Kotak Bank,"[""Im unable to login from the last one month. ...","[quick transactions, friendly, easier]","[user friendly, stands rectified, quick transa...","[app, bank app, kotak app, banking app, good a..."
7,Mobikwik,['It seems there is a serious issue with this ...,"[great update, pdf format, % recommend]","[long time ...., great update .., done .. espi...","[app, mobikwik app, worst app, upi app, good a..."
8,PayTM,"[""Very bad experience. Quite for forever Many ...","[longer time duration, frustrating, error]",[always shows error],"[app, paytm app, app problem, app update, payt..."
9,PayZapp,['app is not opening in android 10 device. my ...,"[other online purchase, taquinical error shows...","[give 1 star, give 0 star, add little amount]","[app, payment app, payzapp app, app recharge, ..."
